# Setup

In [8]:
import csv
import os
import sys

import numpy as np
import pandas as pd
import psycopg2 as pg
import pandas.io.sql as psql

nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
from creds import creds

In [9]:
db = pg.connect("dbname={} user={} password={} host={}".format(creds['dbname'], creds['user'],
                                                               creds['password'], creds['host']))

In [10]:
sql = """
SELECT *
FROM src.review;
"""

df = pd.read_sql_query(sql, db)

In [11]:
df.iloc[0].user_location

'Pittsburgh, Pennsylvania, USA'

In [12]:
df['city'] = None
df['state'] = None
df['country'] = None

def split_location(row):
    parts = row['user_location'].split(',')
    if len(parts) == 0:
        pass
    elif len(parts) == 1:
        row['city'] = parts[-1]
    elif len(parts) == 2:
        row['state'] = parts[-2]
        row['country'] = parts[-1]
    else:
        row['city'] = parts[-3]
        row['state'] = parts[-2]
        row['country'] = parts[-1]
    return row

df = df.apply(lambda row: split_location(row), axis=1)

In [14]:
df.to_csv('review.csv', index=False)